In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer

# **기본 모델 Load**
출처 : https://huggingface.co/unsloth/gemma-2-2b-it

In [ ]:
BASE_MODEL = "unsloth/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/gemma-2b-it",
)

model = AutoModelForCausalLM.from_pretrained(
    "unsloth/gemma-2b-it",
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

# **기본모델 Test**
**모델 성능확인 결과**
1. 한국어로 이야기를 만들 때 내용이 부자연스러움
2. 프롬프트를 조정으로 출력물의 완성도를 높일 수  있으나 자연스러운 스토리라인이 만들어지지 않음
3. 영어 작문은 정상 작동하는 것으로 확인됨

1. 기본 test

In [ ]:
import google.generativeai as genai

# Google API 키 설정 (실제 키로 교체 필요)
#genai.configure(api_key="YOUR_API_KEY")

# Gemma 모델 설정
# model = genai.GenerativeModel('gemma2-2b-it')

def get_user_input():
    """사용자로부터 일상적인 사건을 입력받는 함수"""
    return input("일상적인 사건을 입력해주세요: ")

def create_prompt(event):
    """입력받은 사건을 바탕으로 프롬프트를 생성하는 함수"""
    return f"""
    다음 주제에 대한 이야기를 작성해주세요.
    주제 : "{event}"
    """
def generate_story(prompt):
    """Gemma 모델을 사용하여 이야기를 생성하는 함수"""
    # Use the appropriate method for the Hugging Face model
    # Set max_new_tokens to a value that allows the model to generate enough text
    # Move the input tensors to the same device as the model
    response = model.generate(**tokenizer(prompt, return_tensors="pt").to(model.device), max_new_tokens=512)
    return tokenizer.decode(response[0])

def main():
    event = get_user_input()
    prompt = create_prompt(event)
    story = generate_story(prompt)
    print("\n생성된 이야기:\n")
    print(story)

if __name__ == "__main__":
    main()

2. 프롬프트 수정

In [ ]:
import google.generativeai as genai

# Google API 키 설정 (실제 키로 교체 필요)
#genai.configure(api_key="YOUR_API_KEY")

# Gemma 모델 설정
# model = genai.GenerativeModel('gemma2-2b-it')

def get_user_input():
    """사용자로부터 일상적인 사건을 입력받는 함수"""
    return input("일상적인 사건을 입력해주세요: ")

def create_prompt(event):
    """입력받은 사건을 바탕으로 프롬프트를 생성하는 함수"""
    return f"""
    다음 주제에 대한 이야기를 작성해주세요.
    주제 : "{event}"
    언어 : 한국어
    분량 : 200단어 내외
    """
def generate_story(prompt):
    """Gemma 모델을 사용하여 이야기를 생성하는 함수"""
    # Use the appropriate method for the Hugging Face model
    # Set max_new_tokens to a value that allows the model to generate enough text
    # Move the input tensors to the same device as the model
    response = model.generate(**tokenizer(prompt, return_tensors="pt").to(model.device), max_new_tokens=512)
    return tokenizer.decode(response[0])

def main():
    event = get_user_input()
    prompt = create_prompt(event)
    story = generate_story(prompt)
    print("\n생성된 이야기:\n")
    print(story)

if __name__ == "__main__":
    main()

3. 영어기반 작문유도 프롬프트 수정

In [ ]:
import google.generativeai as genai

# Google API 키 설정 (실제 키로 교체 필요)
#genai.configure(api_key="YOUR_API_KEY")

# Gemma 모델 설정
# model = genai.GenerativeModel('gemma2-2b-it')

def get_user_input():
    """사용자로부터 일상적인 사건을 입력받는 함수"""
    return input("일상적인 사건을 입력해주세요: ")

def create_prompt(event):
    """입력받은 사건을 바탕으로 프롬프트를 생성하는 함수"""
    return f"""
    다음 주제에 대한 이야기를 작성해주세요.
    주제 : "{event}"
    언어 : 한국어

    1. 200단어 내외의 영어 이야기 작성
    2. 영어를 한국어로 번역해서 출력
    """
def generate_story(prompt):
    """Gemma 모델을 사용하여 이야기를 생성하는 함수"""
    # Use the appropriate method for the Hugging Face model
    # Set max_new_tokens to a value that allows the model to generate enough text
    # Move the input tensors to the same device as the model
    response = model.generate(**tokenizer(prompt, return_tensors="pt").to(model.device), max_new_tokens=1000)
    return tokenizer.decode(response[0])

def main():
    event = get_user_input()
    prompt = create_prompt(event)
    story = generate_story(prompt)
    print("\n생성된 이야기:\n")
    print(story)

if __name__ == "__main__":
    main()

# **세가지 종류의 후보 데이터를 활용하여 Gemma Fine-tuning 진행**
1. (AI허브) 다양한 문화콘텐츠 스토리 데이터
- 출처 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71562

2. (huggingface) 네이버 뉴스 요약 데이터
- 출처 : https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko

3. (AI허브) 방송 콘텐츠 대본 요약 데이터
- 출처 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=591

**목표 : 요약본에서 원본을 만들어 내도록 데이터를 학습하여 한국어 생성 성능 향상**

# **데이터1**
1. AI허브 : 다양한 문화콘텐츠 스토리 데이터 (storyline -> script)

- 데이터1 전처리

In [ ]:
import os
import zipfile

def unzip_all(dir, out_dir):
  for f in os.listdir(dir):
    if f.endswith(".zip"):
      f_dir = os.path.join(dir, f)
      with zipfile.ZipFile(f_dir, 'r') as zip_ref:
        zip_ref.extractall(out_dir)
      print(f"{f} 압축 해제 완료")

dir_path = "/content/drive/MyDrive/story_data/Training/Labeled"
out_path = "/content/drive/MyDrive/story_data/data_all"
unzip_all(dir_path, out_path)
dir_path = "/content/drive/MyDrive/story_data/Validation/Labeled"
out_path = "/content/drive/MyDrive/story_data/data_all"
unzip_all(dir_path, out_path)

In [ ]:
import os
import json

# JSON 파일이 있는 디렉토리 경로 설정
dir_path = '/content/drive/MyDrive/story_data/data_all'

# 모든 처리된 데이터를 저장할 리스트
all_processed_data = []

# 디렉토리 내 모든 JSON 파일 처리
for filename in os.listdir(dir_path):
    if filename.endswith('.json'):
        input_file_path = os.path.join(dir_path, filename)

        with open(input_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 각 파일의 데이터 처리
        for story in data["units"]:
            input_text = story['storyline']
            output_text = ' '.join([script['content'] for script in story['story_scripts']])
            all_processed_data.append({'input': input_text, 'output': output_text})

# 처리된 전체 데이터 확인
print(f"총 처리된 데이터 수: {len(all_processed_data)}")

# QLora fine-tuning을 위한 데이터 형식으로 변환
qlora_data = [
    {
        "prompt": f" 다음 상황을 바탕으로 이야기를 만들어주세요: {item['input']}",
        "completion": item['output']
    }
    for item in all_processed_data
]

# QLora 데이터를 JSON 파일로 저장
output_file_path = '/content/drive/MyDrive/story_data/qlora_finetuning_data.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(qlora_data, f, ensure_ascii=False, indent=2)


In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['completion'])):
        prompt_list.append(r"""<bos><start_of_turn>user
        {}<end_of_turn>
        <start_of_turn>model
        {}<end_of_turn><eos>""".format(example['prompt'][i], example['completion'][i]))
    return prompt_list

In [ ]:
from datasets import load_dataset

dataset = load_dataset('json', data_files='/content/drive/MyDrive/story_data/qlora_finetuning_data.json')

train_data = dataset['train']

In [ ]:
train_data[0]

- 데이터1 학습

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model


lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

BASE_MODEL = "unsloth/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL,
                                             quantization_config=bnb_config,
                                             device_map='auto',
                                             )
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'


# Trainer 초기화
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="/content/drive/MyDrive/story_data/outputs_unsloth",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        weight_decay = 0.01,
        push_to_hub=False,
        report_to='none',

        lr_scheduler_type = "linear",
        seed = 777,
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)
 # 학습 실행
trainer.train()

- 데이터1 결과확인

In [ ]:
ADAPTER_MODEL = "/content/drive/MyDrive/story_data/outputs_unsloth/checkpoint-3000"

In [ ]:
# 학습된 weight
!ls -alh /content/drive/MyDrive/story_data/outputs_unsloth/checkpoint-3000

In [ ]:
BASE_MODEL = "unsloth/gemma-2b-it"
model_f1 = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model_f1 = PeftModel.from_pretrained(model_f1, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model_f1 = model_f1.merge_and_unload()

In [ ]:
import google.generativeai as genai

# Google API 키 설정 (실제 키로 교체 필요)
#genai.configure(api_key="YOUR_API_KEY")

# Gemma 모델 설정
# model = genai.GenerativeModel('gemma2-2b-it')

def get_user_input():
    """사용자로부터 일상적인 사건을 입력받는 함수"""
    return input("일상적인 사건을 입력해주세요: ")

def create_prompt(event):
    """입력받은 사건을 바탕으로 프롬프트를 생성하는 함수"""
    return f"""
    다음 주제에 대한 이야기를 작성해주세요.
    주제 : "{event}"
    """
def generate_story(prompt):
    """Gemma 모델을 사용하여 이야기를 생성하는 함수"""
    # Use the appropriate method for the Hugging Face model
    # Set max_new_tokens to a value that allows the model to generate enough text
    # Move the input tensors to the same device as the model
    response = model_f1.generate(**tokenizer(prompt, return_tensors="pt").to(model_f1.device), max_new_tokens=512)
    return tokenizer.decode(response[0])

def main():
    event = get_user_input()
    prompt = create_prompt(event)
    story = generate_story(prompt)
    print("\n생성된 이야기:\n")
    print(story)

if __name__ == "__main__":
    main()

- 부자연스러운 형태가 남아있으나 기존 대비 자연스러운 스토리 전개가 확인됨. 스크립트 형식으로 출력.

# **학습데이터2**
1. Naver new summarization (summarization -> document)

- 데이터2 전처리

In [ ]:
from datasets import load_dataset
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

In [ ]:
def generate_prompt_news(example):
    prompt_list = []
    for i in range(len(example['summary'])):
        prompt_list.append(r"""<bos><start_of_turn>user
        다음 상황을 바탕으로 이야기를 만들어주세요 : {}<end_of_turn>
        <start_of_turn>model
        {}<end_of_turn><eos>""".format(example['summary'][i], example['document'][i]))
    return prompt_list

In [ ]:
train_data = dataset['train']

In [ ]:
print(generate_prompt_news(train_data[:1])[0])

- 데이터2 학습

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

BASE_MODEL = "unsloth/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL,
                                             quantization_config=bnb_config,
                                             device_map='auto',
                                             )
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'


# Trainer 초기화
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="/content/drive/MyDrive/story_data/outputs_news_unsloth",
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        weight_decay = 0.01,
        push_to_hub=False,
        report_to='none',

        lr_scheduler_type = "linear",
        seed = 777,
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt_news,
)

# 학습 실행
trainer.train()

In [ ]:
 ADAPTER_MODEL = "/content/drive/MyDrive/story_data/outputs_news_unsloth/checkpoint-5000"

- 데이터2 결과확인

In [ ]:
# 학습된 weight
!ls -alh /content/drive/MyDrive/story_data/outputs_news_unsloth/checkpoint-5000

In [ ]:
BASE_MODEL = "unsloth/gemma-2b-it"
model_f2 = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model_f2 = PeftModel.from_pretrained(model_f2, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)
model_f2 = model_f2.merge_and_unload()

In [ ]:
import google.generativeai as genai

# Google API 키 설정 (실제 키로 교체 필요)
#genai.configure(api_key="YOUR_API_KEY")

# Gemma 모델 설정
# model = genai.GenerativeModel('gemma2-2b-it')

def get_user_input():
    """사용자로부터 일상적인 사건을 입력받는 함수"""
    return input("일상적인 사건을 입력해주세요: ")

def create_prompt(event):
    """입력받은 사건을 바탕으로 프롬프트를 생성하는 함수"""
    return f"""
    다음 주제에 대한 뉴스를 작성해주세요.
    주제 : "{event}"
    """
def generate_story(prompt):
    """Gemma 모델을 사용하여 이야기를 생성하는 함수"""
    # Use the appropriate method for the Hugging Face model
    # Set max_new_tokens to a value that allows the model to generate enough text
    # Move the input tensors to the same device as the model
    response = model_f2.generate(**tokenizer(prompt, return_tensors="pt").to(model_f2.device), max_new_tokens=512)
    return tokenizer.decode(response[0])

def main():
    event = get_user_input()
    prompt = create_prompt(event)
    story = generate_story(prompt)
    print("\n생성된 이야기:\n")
    print(story)

if __name__ == "__main__":
    main()

- Fine tuning이 잘 되지 않은 것으로 보임

# **학습데이터3**
3. AI허브 : 방송 콘텐츠 대본 요약 데이터 (summary -> passage)

- 데이터3 전처리

In [ ]:
import os
import zipfile

def unzip_all(dir, out_dir):
  """
  지정된 디렉토리에 있는 모든 zip 파일을 압축 해제합니다.

  Args:
    디렉토리_경로: 압축 해제할 zip 파일이 있는 디렉토리 경로입니다.
  """
  for f in os.listdir(dir):
    if f.endswith(".zip"):
      f_dir = os.path.join(dir, f)
      with zipfile.ZipFile(f_dir, 'r') as zip_ref:
        zip_ref.extractall(out_dir)
      print(f"{f} 압축 해제 완료")

dir_path = "/content/drive/MyDrive/broadcast"
out_path = "/content/drive/MyDrive/broadcast"

unzip_all(dir_path, out_path)


In [ ]:
import os
import json

# JSON 파일이 있는 디렉토리 경로 설정
dir_path = '/content/drive/MyDrive/broadcast'

# 모든 처리된 데이터를 저장할 리스트
all_processed_data = []

# 디렉토리 내 모든 JSON 파일 처리
for dirname1 in os.listdir(dir_path):
    for dirname2 in os.listdir(os.path.join(dir_path, dirname1)):
        for filename in os.listdir(os.path.join(dir_path, dirname1,dirname2)):
            if filename.endswith('.json'):
                input_file_path = os.path.join(dir_path, dirname1, dirname2, filename)
                with open(input_file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                # 각 파일의 데이터 처리
                input_text = data["Annotation"]["Summary1"]
                output_text = data["Meta"]["passage"]
                all_processed_data.append({'input': input_text, 'output': output_text})

# 처리된 전체 데이터 확인
print(f"총 처리된 데이터 수: {len(all_processed_data)}")

# QLora fine-tuning을 위한 데이터 형식으로 변환
qlora_data = [
    {
        "prompt": f" 다음 상황을 바탕으로 이야기를 만들어주세요: {item['input']}",
        "completion": item['output']
    }
    for item in all_processed_data
]

# QLora 데이터를 JSON 파일로 저장
output_file_path = '/content/drive/MyDrive/broadcast/qlora_finetuning_data.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(qlora_data, f, ensure_ascii=False, indent=2)

print(f"QLora fine-tuning 데이터가 {output_file_path}에 저장되었습니다.")

- 데이터3 학습

In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['completion'])):
        prompt_list.append(r"""<bos><start_of_turn>user
        {}<end_of_turn>
        <start_of_turn>model
        {}<end_of_turn><eos>""".format(example['prompt'][i], example['completion'][i]))
    return prompt_list

from datasets import load_dataset

dataset = load_dataset('json', data_files='/content/drive/MyDrive/broadcast/qlora_finetuning_data.json')

train_data = dataset['train']

In [ ]:
# QLoRA 설정
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
train_data[0]

In [ ]:
# 모델 로드
BASE_MODEL = "unsloth/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL,
                                             quantization_config=bnb_config,
                                             device_map='auto',
                                             )
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="/content/drive/MyDrive/broadcast/output_3",
        max_steps=2000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=50,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)
# 학습 실행
trainer.train()

- 데이터3 결과확인

In [ ]:
 ADAPTER_MODEL = "/content/drive/MyDrive/broadcast/output_3/checkpoint-500"

In [ ]:
BASE_MODEL = "unsloth/gemma-2b-it"
model_f3 = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model_f3 = PeftModel.from_pretrained(model_f3, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model_f3 = model_f3.merge_and_unload()

In [ ]:
import google.generativeai as genai

# Google API 키 설정 (실제 키로 교체 필요)
#genai.configure(api_key="YOUR_API_KEY")

# Gemma 모델 설정
# model = genai.GenerativeModel('gemma2-2b-it')

def get_user_input():
    """사용자로부터 일상적인 사건을 입력받는 함수"""
    return input("일상적인 사건을 입력해주세요: ")

def create_prompt(event):
    """입력받은 사건을 바탕으로 프롬프트를 생성하는 함수"""
    return f"""
    다음 주제에 대한 이야기를 작성해주세요.
    주제 : {event}
    """
def generate_story(prompt):
    """Gemma 모델을 사용하여 이야기를 생성하는 함수"""
    # Use the appropriate method for the Hugging Face model
    # Set max_new_tokens to a value that allows the model to generate enough text
    # Move the input tensors to the same device as the model
    response = model_f3.generate(**tokenizer(prompt, return_tensors="pt").to(model_f3.device), max_new_tokens=512)
    return tokenizer.decode(response[0])

def main():
    event = get_user_input()
    prompt = create_prompt(event)
    story = generate_story(prompt)
    print("\n생성된 이야기:\n")
    print(story)

if __name__ == "__main__":
    main()

- Fine tuning이 잘 되지 않은 것으로 보임

# **결과요약**
**1. (AI허브) 다양한 문화콘텐츠 스토리 데이터**

- 기본모형 대비 자연스러운 결과를 보여줌. 스크립트 형식으로 출력.

**2. (huggingface) 네이버 뉴스 요약 데이터**

- 뉴스형식으로 출력을 시작하지만 정상적인 output이 출력되지 않음.

**3. (AI허브) 방송 콘텐츠 대본 요약 데이터**

- 정상 출력되지 않음. 방송 콘텐츠의 텍스트 형식에 대한 추가 전처리 과정이 필요.

# ** model_f1을 기준으로 추가 결과 확인**

In [ ]:
ADAPTER_MODEL = "/content/drive/MyDrive/story_data/outputs_unsloth/checkpoint-3000"

In [ ]:
BASE_MODEL = "unsloth/gemma-2b-it"
model_f1 = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model_f1 = PeftModel.from_pretrained(model_f1, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model_f1 = model_f1.merge_and_unload()

In [ ]:
import google.generativeai as genai

# Google API 키 설정 (실제 키로 교체 필요)
#genai.configure(api_key="YOUR_API_KEY")

# Gemma 모델 설정
# model = genai.GenerativeModel('gemma2-2b-it')

def get_user_input():
    """사용자로부터 일상적인 사건을 입력받는 함수"""
    return input("일상적인 사건을 입력해주세요: ")

def create_prompt(event):
    """입력받은 사건을 바탕으로 프롬프트를 생성하는 함수"""
    return f"""
    다음 주제에 대한 이야기를 작성해주세요.
    주제 : "{event}"
    """
def generate_story(prompt):
    """Gemma 모델을 사용하여 이야기를 생성하는 함수"""
    # Use the appropriate method for the Hugging Face model
    # Set max_new_tokens to a value that allows the model to generate enough text
    # Move the input tensors to the same device as the model
    response = model_f1.generate(**tokenizer(prompt, return_tensors="pt").to(model_f1.device), max_new_tokens=512)
    return tokenizer.decode(response[0])

def main():
    event = get_user_input()
    prompt = create_prompt(event)
    story = generate_story(prompt)
    print("\n생성된 이야기:\n")
    print(story)

if __name__ == "__main__":
    main()

In [ ]:
if __name__ == "__main__":
    main()

# **결론**
- Fine Tuning을 통해 기존보다 자연스러운 한국어 이야기를 구현하게 되었지만, 여전히 입력 텍스트에 대한 맥락 이해가 부족하고 한국어 문법에서 오류를 보이는 경우가 많다.

- 대규모 학습을 통해 개선이 가능할 듯하다.